# Reading data from the Netcdf database using the netCDF4 module

So far, two  database types are supported by the current version of SHyFT:
- Statkraft internal database ("orchestration"); very limited access
- NETCDF database ("orchestration2")

Here we focus on the NETCDF database!


In [6]:
from netCDF4  import Dataset

In [27]:
# path to the dataset files
import os
data_path = os.path.abspath(os.path.realpath('../atnsjoen_catchment/data/'))


####The database exists of three files:
- Meteorologiccal observations
- Discharge observatioons
- Land surface data

In [32]:
for filename in os.listdir(data_path):
    print filename

atnsjoen_cell_info.nc
atnsjoen_discharge.nc
atnsjoen_met.nc


##Meteorological observations

In [33]:
fn_met = "atnsjoen_met.nc"

# opening the file
with Dataseet(fn_met))


In [22]:
a='/home/felix/.mnts/net/nunatak/mn/nunatak/hycamp/project_shyft/shyft_course'



In [24]:
!ls a


ls: cannot access a: No such file or directory
